In [ ]:
import pygmsh
import trimesh
import capytaine as cpt
import pyvista as pv
import numpy as np

with pygmsh.occ.Geometry() as geom:
    #geom.characteristic_length_max = 0.2
    cyl1 = geom.add_cylinder([0.01, 0, 0], [0, 0, 0.1],  2)
    cyl2 = geom.add_cylinder([0.01, 0, 0.1], [0, 0, 0.1],  2)
    cyl3 = geom.add_cylinder([0.01, 0, 0.2], [0, 0, 0.1],  2)
    geom.boolean_union([cyl1, cyl2, cyl3])
    gmsh_mesh = geom.generate_mesh(dim=2)
mesh_pv = pv.wrap(gmsh_mesh)

In [ ]:
# Convert the pygmsh mesh to a trimesh mesh
surf = mesh_pv.extract_surface()
surf.save('test.stl')
tri_mesh = trimesh.load_mesh('test.stl')

# Compute the center of mass with trimesh
center_of_mass = tri_mesh.centroid

In [ ]:
np.round(center_of_mass,2)

In [ ]:
import meshio
#mesh = meshio.read('test.ply')
mesh = cpt.load_mesh('test.stl')
#mesh = cpt.mesh_sphere(radius=1.0, center=(0, 0, -2), name="my sphere")
from capytaine.meshes.predefined.cylinders import mesh_vertical_cylinder
cyl1 = cpt.VerticalCylinder(
        length=0.1, radius=1,
        center=(0,0,0),
        nr=100, nx=100, ntheta=10,
    )
cyl2 = cpt.VerticalCylinder(
        length=0.1, radius=1,
        center=(0,0,0.05),
        nr=100, nx=100, ntheta=10,
    )
cyl3 = cpt.VerticalCylinder(
        length=0.1, radius=1,
        center=(0,0,0.15),
        nr=100, nx=100, ntheta=10,
    )
body = cyl1+cyl2+cyl3
body.rotation_center = np.array([0, 0, 0])
body.center_of_mass = np.array([0,0,0.16])
body.add_all_rigid_body_dofs()
# body = cpt.FloatingBody(mesh, 
#                         dofs=cpt.rigid_body_dofs(rotation_center=(0,0,1)),
#                         center_of_mass = center_of_mass)
hydrostatics = body.compute_hydrostatics(rho=1023.6)

In [ ]:
body.immersed_part().volume

In [ ]:
#pv.set_jupyter_backend('static')

# # Create a plotter object and render the mesh
# plotter = pv.Plotter(notebook=False)
# plotter.add_mesh(mesh)
# plotter.show()